In [6]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten

import random
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re

In [2]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB','Class']
# dataset = dataset.loc[:,cols]
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class
dataset.sample(10)

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
1423,84923,1199.2190,469.3299,232.0247,0.8692,328.8272,0.9816,86519,0.6558,2.0228,...,21.2477,22.4891,22.9599,-1.1784,-0.9764,-0.7704,3.8433,3.2281,2.6371,Siirt_Pistachio
871,85845,2041.5470,490.6570,237.4381,0.8751,330.6074,0.8941,96011,0.7345,2.0665,...,21.4586,22.7930,27.0073,-0.7880,-0.5331,-0.5011,3.0151,2.8876,4.2760,Kirmizi_Pistachio
96,84880,1458.6570,475.3602,246.4661,0.8551,328.7439,0.9233,91934,0.7060,1.9287,...,20.5703,23.0201,21.6779,-0.7622,-0.7448,-0.2291,3.6905,3.4721,3.5018,Kirmizi_Pistachio
942,54601,1348.1689,421.2827,182.4424,0.9014,263.6667,0.8768,62274,0.5580,2.3091,...,15.4110,16.1671,18.2462,-0.2044,-0.0904,0.1528,2.7345,3.1792,3.2125,Kirmizi_Pistachio
239,68626,1407.5320,448.5386,198.9794,0.8962,295.5966,0.9471,72456,0.7662,2.2542,...,24.4091,25.6984,28.5108,-0.8011,-0.6479,-0.6135,2.5193,2.4528,2.4083,Kirmizi_Pistachio
718,67659,1446.4280,396.5165,226.1531,0.8214,293.5066,0.9134,74071,0.7631,1.7533,...,23.0826,24.0397,24.7538,-0.4170,-0.4411,-0.3590,2.2057,2.3340,2.6809,Kirmizi_Pistachio
881,41269,1338.3800,370.0265,153.0344,0.9105,229.2277,0.8900,46369,0.6439,2.4179,...,18.0830,19.2815,19.6241,-0.3768,-0.2495,-0.0783,2.1682,2.0709,2.2075,Kirmizi_Pistachio
70,81164,1222.9310,442.9454,242.1429,0.8374,321.4673,0.9392,86419,0.7543,1.8293,...,21.4675,23.0536,23.5558,-0.9706,-0.7575,-0.5712,3.3370,3.0248,3.1850,Kirmizi_Pistachio
1065,67270,1084.4500,425.1971,204.9928,0.8761,292.6616,0.9711,69272,0.7504,2.0742,...,24.1879,26.4544,27.2019,-0.8846,-0.6758,-0.4558,2.9020,2.5082,2.3304,Kirmizi_Pistachio
409,61365,1042.7690,416.7724,190.9898,0.8888,279.5216,0.9785,62716,0.7165,2.1822,...,15.3367,15.5665,16.7052,-0.3976,-0.1698,0.4866,3.8909,4.1458,5.1740,Kirmizi_Pistachio


In [3]:
dataset = shuffle(dataset)

In [4]:
lis = []
for i in range(28):
    lis.append(27-i)

In [5]:
def FReIndex(colIndex,reIndex):
    dirindex = {}
    lisindex = []
    for i in range(len(colIndex)):
        lisindex.append(i)
    dirindex = dict(zip(lisindex,colIndex))
    colIndex = []
    for i in range(len(reIndex)):
        for key in dirindex:
            if int(key) == int(reIndex[i]):
                colIndex.append(dirindex[key])
                break
    print(colIndex)
    return colIndex
    

In [6]:
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
# lisindex = []
# for i in range(len(dataset.columns)):
#     lisindex.append(i)
# dataset = dataset.reindex(lisindex)
colIndex = ['Area','Minor_Axis', 'Eccentricity',
            'Eqdiasq','Convex_Area', 'Aspect_Ratio',
            'Compactness', 'Shapefactor_1', 
            'Shapefactor_3']
#colIndex = FReIndex(colIndex,lis)
dataset = pd.DataFrame(dataset,columns=colIndex)
dataset


,Area,Minor_Axis,Eccentricity,Eqdiasq,Convex_Area,Aspect_Ratio,Compactness,Shapefactor_1,Shapefactor_3
987,83711,256.2626,0.8458,326.4723,98431,1.8745,0.6796,0.0057,0.4619
330,61836,238.2245,0.7779,280.5923,75468,1.5913,0.7402,0.0061,0.5479
1404,89823,264.6333,0.8090,338.1807,96010,1.7011,0.7512,0.0050,0.5643
1407,98290,303.0911,0.7207,353.7608,105919,1.4426,0.8091,0.0044,0.6546
1971,77788,245.5920,0.8038,314.7106,80677,1.6811,0.7622,0.0053,0.5810
...,...,...,...,...,...,...,...,...,...
1607,91872,264.0024,0.8143,342.0162,97708,1.7228,0.7520,0.0050,0.5655
1039,75516,245.7620,0.8023,310.0806,82546,1.6753,0.7531,0.0055,0.5672
1434,94328,257.9542,0.8351,346.5576,95643,1.8179,0.7390,0.0050,0.5461
1812,92155,244.4703,0.8646,342.5425,94256,1.9899,0.7041,0.0053,0.4958


In [7]:
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [8]:

import os,sys
import shutil

shutil.rmtree('models')  
os.mkdir('models')

In [9]:
#定义模型

sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)

lstm_layers = [1,2,3,4,5]
dense_layers = [1,2,3,4,5,6]
units = [16,32,64,128]
dropout = [0.05,0.1,0.15,0.25]
Batch_size = [32,64,128]
optimizer = ['adam',sgd]
for the_batch_size in Batch_size:
    for the_dropout in dropout:
        for the_optimizer in optimizer:
            for the_dense_layers in dense_layers:
                for the_lstm_layers in lstm_layers:
                    for the_units in units:
                        sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
                        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=random.randint(10,100))
                        X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
                        X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
                        model = Sequential()
                        # model.build(input_shape=(277,277,2))
                        #print(model.summary())
                        #model.add(SpatialDropout1D(0.2))
                        model.add(LSTM(the_units ,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences = True))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        # #第二层
                        for i in range(the_lstm_layers):
                            model.add(LSTM(the_units,return_sequences=True))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())

                        model.add(LSTM(the_units))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        #全连接层
                        for i in range(the_dense_layers):
                            model.add(Dense(the_dense_layers,activation='relu'))
                            model.add(Dropout(the_dropout))
                            
                        # model.add(Flatten()) 
                        
                        
                        model.add(Dense(2, activation='softmax'))
                        
                        #sgd = SGD(learning_rate=0.01, momentum=0.9 , decay=0.1, nesterov=False)
                        
                        # learning_rate = 0.1
                        # decay = 0.001
                        # epochs = 50
                        # batch_size = 64
                        
                        
                        model.compile(  loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                                        optimizer=the_optimizer, metrics=['categorical_accuracy'])
                        print(model.summary())

                        epochs = 100
                        batch_size = the_batch_size
                        if(the_optimizer == sgd):
                            the_optimizer = 'sgd'
                        filepath = './models/{categorical_accuracy:.4f}_{epoch:02d}_'+f'dropout_{the_dropout}_batch_size_{the_batch_size}_optimizer_{the_optimizer}_dense_layers_{the_dense_layers}_lstm_layers_{the_lstm_layers}_unit_{the_units}.h5'
                        checkpoint = ModelCheckpoint(
                                            filepath=filepath,
                                            save_weights_only=False,
                                            monitor='categorical_accuracy',
                                            mode='max',
                                            save_best_only=True)
                        history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1,
                                            #callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
                                            callbacks=checkpoint)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 16)             1664      
_________________________________________________________________
dropout (Dropout)            (None, 1, 16)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 16)             64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 16)             2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 16)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 16)             64        
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                2

KeyboardInterrupt: 

In [9]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/0.8597_99_dropout_0.05_batch_size_32_optimizer_adam_dense_layers_1_lstm_layers_1_unit_16.h5')

In [10]:
dataset = pd.read_csv('test.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']

lis = []
for i in range(28):
    lis.append(27-i)

Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
# lisindex = []
# for i in range(len(dataset.columns)):
#     lisindex.append(i)
# dataset = dataset.reindex(lisindex)
colIndex = ['Area','Minor_Axis', 'Eccentricity',
            'Eqdiasq','Convex_Area', 'Aspect_Ratio',
            'Compactness', 'Shapefactor_1', 
            'Shapefactor_3']
#colIndex = FReIndex(colIndex,lis)
dataset = pd.DataFrame(dataset,columns=colIndex)

X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#进行预测 make a prediction
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.9, random_state=random.randint(10,100))
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
# print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
yhat = best_model.predict(X_test)
# print(yhat.shape)
print(yhat)
print(Y_test)
sorce = 0
for i in range(len(Y_test)):
    sorce += roc_auc_score(Y_test[i], yhat[i])

print('ACC:',sorce/len(Y_test))
#拆分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=random.randint(10,100))
# X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

[[7.7074190e-05 9.9992287e-01]
 [9.5783162e-01 4.2168371e-02]
 [8.8447960e-05 9.9991155e-01]
 [9.5783162e-01 4.2168371e-02]
 [9.5783162e-01 4.2168371e-02]
 [9.5783162e-01 4.2168371e-02]
 [3.7874389e-04 9.9962127e-01]
 [6.3452451e-04 9.9936551e-01]
 [1.2191693e-04 9.9987805e-01]
 [8.4994914e-04 9.9915004e-01]
 [2.3041708e-04 9.9976963e-01]
 [9.5783162e-01 4.2168371e-02]
 [9.5783162e-01 4.2168371e-02]
 [3.0745062e-01 6.9254935e-01]
 [1.3609251e-01 8.6390746e-01]
 [6.9754067e-05 9.9993026e-01]
 [2.5877409e-04 9.9974126e-01]]
[[0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]
ACC: 0.9411764705882353
